データの探索
===

大量のデータの中から目的の値を見つけるには？

### Pythonにおけるデータの探索

In [1]:
import random
# ランダムなリストを生成
random.seed(5)
my_array = [random.randint(0, 100) for i in range(15)]
my_array

[79, 32, 94, 45, 88, 94, 83, 67, 3, 59, 99, 31, 83, 6, 20]

In [2]:
# in 演算子を用いることで見つける
print(32 in my_array)
print(50 in my_array)
print(my_array.index(32))

True
False
1


### 線形探索
先頭から順番に探していく。

In [3]:
def linear_search(array, target):
    for v in array:
        if target == v:
            return True
    return False

print(linear_search(my_array, 32))

print(linear_search(my_array, 50))


True
False


データの数を$n$とすると、平均的には $\mathcal{O}(\frac{n}{2})$ の比較が考えられる。
当然最悪の場合は $\mathcal{O}(n)$ の計算量となる。

僕がプログラムを書く場合は、たいていこのパターン。

### 二分探索
配列がソートされているならば、もっと早く見つけることができる。

二分探索$O(\log n)$ < 線形探索$O(n)$


In [4]:
import bisect
my_array.sort()
print(bisect.bisect(my_array, 32))

5


In [5]:
def my_bisect_right(a, x, lo=0, hi=None):
    if lo < 0:
        raise ValueError('lo must be non-negative')
    if hi is None:
        hi = len(a)
    while lo < hi:
        mid = (lo+hi)//2
        if x < a[mid]: hi = mid
        else: lo = mid+1
    return lo


データが見つかればOKだが、見つからないケースもある。
見つからない場合は最後の到達点がそのデータの挿入ポイントでもある。

２分探索は配列がソートされている前提の探索方法であり、データを挿入すると中身が変わる。
探索自体は$\mathcal{O}(\log n)$でできるが、挿入には$\mathcal{O}(n)$の計算量がかかり、実質的に$\mathcal{O}(n \log n)$の計算量となる。

挿入時も$\mathcal{O}(\log n)$の計算量で済むデータ構造が欲しい。

## 探索のためのデータ構造

データを一直線に並んだ配列ではなく木構造で保持すると、探索と挿入に柔軟に対応できる。

### 用語

**木**(tree)は、**ノード**(node)と**枝**(branch)からなる。
もっとも上位にあるノードは**根**(root)と呼ばれる。
個ノードを持たないノードはもっとも下位に位置することになり、これを**葉**(leaf)という。

根からあるノードまでの距離を**深さ**(depth)と言う。
根からもっとも遠い葉までの距離を木の**高さ**(height)という。

あるノードにはいくつのこのノードがあっても木構造上問題ないが、2つまでとしている場合を**二分木**(binary tree)と呼ぶ。
さらに、二分木のうち高さがどれも同じになっているものを**完全二分木**(complete binary tree)という。

二分木の中でも、ノードの親子関係と値の大小関係があり、全てのノードがこの関係を満たしている二分木を**二分探索木**(binary search tree)という。



In [6]:
class Node:

    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

    def __str__(self):
        # Nodeクラスのインスタンスを文字列表現にする
        left = f'[{self.left.value}]' if self.left else '[]'
        right = f'[{self.right.value}]' if self.right else '[]'
        return f'{left} <- {self.value} -> {right}'


class BinarySearchTree:
    def __init__(self):
        self.nodes = []

    def add_node(self,value):
        node = Node(value)
        if self.nodes:
            parent, direction = self.find_parent(value)
            if direction == 'left':
                parent.left = node
            else:
                parent.right = node
        self.nodes.append(node)

    def find_parent(self, value):
        node = self.nodes[0]
        # node が None になるまでループを回す
        while node:
            p = node # 戻り値の候補(親かもしれない)として取っておく
            if p.value == value:
                raise ValueError('すでにある値と同じ値を格納することはできません')
            if p.value > value:
                direction = 'left'
                node = p.left
            else:
                direction = 'right'
                node = p.right
        return p, direction

In [7]:
btree = BinarySearchTree()
for v in [10, 20, 12, 4, 3, 9, 30]:
    btree.add_node(v)

for node in btree.nodes:
    print(node)

[4] <- 10 -> [20]
[12] <- 20 -> [30]
[] <- 12 -> []
[3] <- 4 -> [9]
[] <- 3 -> []
[] <- 9 -> []
[] <- 30 -> []


In [8]:
# この例だと、バランスの偏りが極端であるため配列を走査するのと同じ計算量となってしまう
btree = BinarySearchTree()
for v in sorted([10, 20, 12, 3, 4, 9, 30]):
    btree.add_node(v)

for node in btree.nodes:
    print(node)


[] <- 3 -> [4]
[] <- 4 -> [9]
[] <- 9 -> [10]
[] <- 10 -> [12]
[] <- 12 -> [20]
[] <- 20 -> [30]
[] <- 30 -> []


### 木のバランスが大事
２分探索木を使ってデータの探索や挿入で、$O(log n)$の計算量になるのは、２分探索木の左右のバランスが取れている場合に限る。
一般に左右バランスが取れている木を**平衡木**(balanced tree)と言う。



### ヒープ構造
データの最小値を取り出したい時に便利なデータ構造。
データが昇順にソードされた配列を考えると、先頭から順番に取ってくればそれは最小値である。
しかし、取り出しもにではなく、挿入も考えると挿入点を探す必要がある。
ソートされていることを考えると二分探索を行うことができ、挿入点の探索は計算量は$\mathcal{O}(\log n)$で良いが、挿入するすることを考えると$\mathcal{O}(n)$となる。

常に最小値だけわかれば良い場合、「先頭は最小値」であることだけを補償し、他はソートされている必要がないため計算量を削減できる。

#### ヒープの更新

最小値の取り出しは、ルートを取り出せば良いだけであるため、$\mathcal{O}(1)$となる。
その後、木構造の修復に時間がかかる。
しかし、ヒープは完全二分木であるため、頂点の数を$n$とした時、深さは$\log(n)$程度となる。
大小関係の矛盾を解決するためのステップが、根から葉まで到達すればよいので、$\mathcal{O}(\log n)$となる。

#### ヒープソート

ヒープは常に最小値を教えてくれるため、これを用いて配列をソートすることができる。




In [9]:
import heapq

# ヒープソート
def heap_sort(array):
    heap = []
    for v in array:
        heapq.heappush(heap, v)
    return [heapq.heappop(heap) for _ in range(len(heap))]

In [10]:
my_array = [random.randint(0, 100) for i in range(15)]
print(my_array)
print(heap_sort(my_array))

[14, 47, 60, 31, 48, 69, 13, 73, 31, 1, 93, 27, 52, 35, 23]
[1, 13, 14, 23, 27, 31, 31, 35, 47, 48, 52, 60, 69, 73, 93]


計算量は、入力サイズを$n$としたき、ヒープ作成とその後にヒープ利用の合計となる。
ヒープの作成は$\mathcal{O}(n)$でできる。
各ステップではヒープから値を取り出すため、ヒープの更新が必要となる。これは、$\mathcal{O}(\log n)$の計算量。
要素が$n$個あるため、$\mathcal{O}(n \log n)$となる。
ヒープ作成の$\mathcal{O}(n)$よりも大きいため、ヒープソートの計算量は$\mathcal{O}(n \log n)$となる。

マージソートやクイックソートと同じ計算量となる。

## ハッシュを使った探索
### Pythonの辞書型


データのハッシュ値を、データの探索に利用するのが**ハッシュテーブル**である。
ハッシュテーブルの探索にかかる計算量は、データのサイズによらず$\mathcal{O}(1)$の定数時間となる

> どんなに大きな整数値がきても、それを一定の範囲に収まるように変換する方法はあるか？




In [11]:
divmod(301, 100)


(3, 1)

In [12]:
divmod(-999, 100)

(-10, 1)

In [13]:
class HashTable:

    def __init__(self, table_size=100):
        self.data = [[] for i in range(table_size)]
        self.n = table_size

    def get_hash(self, v):
        """オブジェクトのハッシュ値を計算する"""
        return hash(v) % self.n

    def search(self, key):
        i = self.get_hash(key)
        for j, v in enumerate(self.data[i]):
            if v[0] == key:
                return (i, j)
        return (i, -1)

    def set(self, key, value):
        i, j = self.search(key)
        if j != -1:
            self.data[i][j][1] = value
        else:
            self.data[i].append([key, value])

    def get(self, key):
        i, j = self.search(key)
        if j != -1:
            return self.data[i][j][1]
        raise KeyError(f'{key} was not found in this HashTable!')

In [14]:
my_hash_table = HashTable()
my_hash_table.set('taro',10)
my_hash_table.get('taro')

10